In [1]:
#include <iostream>
#include <unistd.h>
#include <stdio.h>
#include <sys/socket.h>
#include <stdlib.h>
#include <netinet/in.h>
#include <string.h>

#define PORT 2020

using namespace std;

#include <nlohmann/json.hpp>

// for convenience
using json = nlohmann::json;

## Импорт и экспорт структуры

Для начала необходимо придумать структуру для хранения. А будет это двусвязный список.

In [2]:
enum TYPE {INTEGER, FLOAT, STRING, BOOL, HOLD, LIST, DOT, TIMES, PLUS};

In [3]:
typedef struct node{
    TYPE type;
    struct {
        int i;
        double f;
        string s;
        vector<node*> args;
    } data;  
    
    node() {
        
    }
} node;

In [4]:
string EnumTypes[] =
{
   "INTEGER", "FLOAT", "STRING","BOOL", "Hold", "List", "Dot", "Times", "Plus" 
};

In [5]:
void show(node &data, int level = 0);

In [6]:
void show(node &data, int level = 0) {
    
    string offset = "";
    for (int i=0; i<level; ++i) offset = offset + "  ";
    
    if (data.type == INTEGER) {
        cout << offset << data.data.i << '\n';
        return;
    }
    
    if (data.type == FLOAT) {
        cout << offset << data.data.f << '\n';
        return;
    }
    
    if (data.type == STRING) {
        cout << offset << data.data.s << '\n';
        return;
    }     
    
    cout << offset << "Type: " << EnumTypes[data.type] << '\n';
    
    level = level + 1;
    for (int i=0; i<data.data.args.size(); ++i) {
        show(*data.data.args[i], level);
    }
    
    return;
}

Теперь надо научить записывать туда дерево и считывать

In [7]:
unsigned constexpr const_hash(char const *input) {
    return *input ?
        static_cast<unsigned int>(*input) + 33 * const_hash(input + 1) :
        5381;
}

In [8]:
unsigned const_hash(string c) {
    return const_hash(c.c_str());
}

In [9]:
node* (*btree)(json &data) = nullptr;
json* (*ftree)(node &data) = nullptr;

In [10]:
btree = [](json &data) {
    node* ptr = new node;
    
    if (data.is_number_integer()) {
        ptr->type = INTEGER;
        ptr->data.i = data.get<int>();
        return ptr;
    }
    
    if (data.is_number_float()) {
        ptr->type = FLOAT;
        ptr->data.f = data.get<double>();
        return ptr;
    }   
    
    if (data.is_string()) {
        ptr->type = STRING;
        ptr->data.s = data.get<string>();
        return ptr;
    }    
    
    string name = data[0].get<string>();
    
    switch(const_hash(name)) {
        case const_hash("Hold"):
            ptr->type = HOLD;
            break;
        case const_hash("Dot"):
            ptr->type = DOT;
            break;
        case const_hash("List"):
            ptr->type = LIST;
            break;
        case const_hash("Plus"):
            ptr->type = PLUS;
            break;
        case const_hash("Times"):
            ptr->type = TIMES;
            break;
    }
    
    if (data.size() < 2) return ptr;
    
    for (int i = 1; i<data.size(); ++i) {
        json obj = data[i];
        
        ptr->data.args.push_back(btree(obj));
        
    }
    
    return ptr;
};

In [18]:
ftree = [](node &data) {
    json* ptr = new json;
    
    if (data.type == INTEGER) {
        *ptr = data.data.i;
        return ptr;
    }
    
    if (data.type == FLOAT) {
        *ptr = data.data.f;
        return ptr;
    }   
    
    if (data.type == STRING) {
        *ptr = data.data.s;
        return ptr;
    }    
    
    ptr->push_back(EnumTypes[data.type]);
    
    for (int i = 0; i<data.data.args.size(); ++i) {
        
       ptr->push_back(*ftree(*data.data.args[i]));
        
    }
    
    return ptr;
};

In [19]:
char text[] = R"(
[
		"Dot",
		[
			"Times",
			2,
			[
				"List",
				[
					"List",
					1,
					0
				],
				[
					"List",
					0,
					1
				]
			]
		],
		[
			"List",
			[
				"List",
				0,
				1
			],
			[
				"List",
				1,
				"u"
			]
		]
]
)";
// Let's parse and serialize JSON
json buffer = json::parse(text);
node* ptr = btree(buffer);
show(*ptr);
json* second = ftree(*ptr);

cout << buffer << '\n';
cout << *second;

Type: Dot
  Type: Times
    2
    Type: List
      Type: List
        1
        0
      Type: List
        0
        1
  Type: List
    Type: List
      0
      1
    Type: List
      1
      u
["Dot",["Times",2,["List",["List",1,0],["List",0,1]]],["List",["List",0,1],["List",1,"u"]]]
["Dot",["Times",2,["List",["List",1,0],["List",0,1]]],["List",["List",0,1],["List",1,"u"]]]